In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import datetime as datetime
import json
import os
import csv
import math
from datetime import timedelta
from openpyxl import load_workbook
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from scipy import stats

In [29]:
def vol_prepro(df6,date_1,date_2,date_3):
    tmp_arr = np.array(list(df6))
    pos_1 = np.where(tmp_arr==date_1)[0][0]
    pos_2 = np.where(tmp_arr==date_2)[0][0]
    pos_3 = np.where(tmp_arr==date_3)[0][0]
    
    df_before = df6.iloc[:,pos_1:pos_2]
    df_after = df6.iloc[:,pos_2:(pos_3+1)]
    
    df_before['mean_before'] = df_before.mean(axis=1)
    df_after['mean_after'] = df_after.mean(axis=1)
    
    y = df_after['mean_after']-df_before['mean_before']
    return y

In [50]:
def spread_prepro(df7,date_1,date_2,date_3):
    tmp_arr = np.array(list(df7))
    pos_1 = np.where(tmp_arr==date_1)[0][0]
    pos_2 = np.where(tmp_arr==date_2)[0][0]
    pos_3 = np.where(tmp_arr==date_3)[0][0]
    
    df_before = df7.iloc[:,pos_1:pos_2]
    df_after = df7.iloc[:,pos_2:(pos_3+1)]
    
    df_before['mean_before'] = df_before.mean(axis=1)
    df_after['mean_after'] = df_after.mean(axis=1)
    
    y = df_after['mean_after'][1::2]-df_before['mean_before'][1::2]
    return y

In [150]:
def regression(res, date_1, date_2, date_3):
        """
        根據date_1, date_2, date_3來切割dataframe
        """
        df1 = res.iloc[:,:248]
        tmp_arr = np.array(list(df1))
        pos_1 = np.where(tmp_arr==date_1)[0][0]
        pos_2 = np.where(tmp_arr==date_2)[0][0]
        pos_3 = np.where(tmp_arr==date_3)[0][0]
        #print("date_1 pos: {}\ndate_2 pos: {}\ndate_3 pos: {}".format(self.pos_1,self.pos_2,self.pos_3))
        
        df_before = df1.iloc[:,pos_1:pos_2]
        df_after = df1.iloc[:,pos_2:(pos_3+1)]
        
        df_before['mean_before'] = df_before.mean(axis=1)
        df_after['mean_after'] = df_after.mean(axis=1)
        df_before['std_before'] = df_before.std(axis=1)
        df_after['std_after'] = df_after.std(axis=1)
        
        """
        prepare OLS 
        """
        y_daily = df_after['std_after'][0::4]-df_before['std_before'][0::4]
        y_weekly = df_after['std_after'][1::4]-df_before['std_before'][1::4]
        y_monthly = df_after['std_after'][2::4]-df_before['std_before'][2::4]
        y_vol = res.iloc[:,257][0::4] # vol週轉率
        y_spread = res.iloc[:,258][0::4] # 賣買價差%
        x1 = df_after['mean_after'][3::4]-df_before['mean_before'][3::4]
        x2 = res.iloc[:,249][0::4] # log(assets)
        x3 = res.iloc[:,252][0::4] # D/A
        x4 = res.iloc[:,254][0::4] # P/B
        x5 = res.iloc[:,256][0::4] # beta
        
        df_final = pd.concat([y_spread,x1,x2,x3,x4,x5],axis=1)
        df_final.columns=['y_spread','x1','x2','x3','x4','x5']
        df_final['y_spread'].fillna(value=df_final['y_spread'].mean(), inplace=True)
        df_final['x1'].fillna(value=df_final['x1'].mean(), inplace=True)
        df_final['x2'].fillna(value=df_final['x2'].mean(), inplace=True)
        df_final['x2'] = np.log(df_final['x2'])
        df_final['x3'].fillna(value=df_final['x3'].mean(), inplace=True)
        df_final['x4'].fillna(value=df_final['x4'].mean(), inplace=True)
        df_final['x5'].fillna(value=df_final['x5'].mean(), inplace=True)
        
        #return df_final.head()
        
        X = df_final.iloc[:,1:]
        Y = df_final.iloc[:,0]

        X = sm.add_constant(X)
        model = sm.OLS(Y,X).fit()
        print(model.summary())

In [151]:
df1 = pd.read_csv("new_data/報酬率_pei/日報酬csv/上市上櫃公司20161101_20171031.csv",index_col="證券代碼")
df2 = pd.read_csv("new_data/資產總額_new/上市櫃20161101-20171031.csv",index_col="公司",thousands=',')
df3 = pd.read_csv("new_data/DA_new/上市櫃2016.csv",index_col="公司")
df4 = pd.read_csv("new_data/PB_new/上市櫃2016.csv",index_col="證券代碼")
df5 = pd.read_csv("new_data/beta_new/上市櫃2016.csv",index_col="證券代碼")
df6 = pd.read_csv("new_data/封存_juu/周轉率new/上市櫃20161101-20171031.csv", index_col="證券代碼")
df7 = pd.read_csv("new_data/spread_pei/ALL20161101_20171031.csv", index_col="證券代碼")
vol = vol_prepro(df6,"2016/11/1","2017/4/28","2017/10/31")
spread = spread_prepro(df7,"2016/11/1","2017/4/28","2017/10/31")

res = pd.concat([df1,df2,df3,df4,df5,vol,spread],axis=1, join_axes=[df1.index])

In [152]:
regression(res,"2016/11/1","2017/4/28","2017/10/31")

                            OLS Regression Results                            
Dep. Variable:               y_spread   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.455
Date:                Sun, 17 Nov 2019   Prob (F-statistic):           0.000512
Time:                        11:51:23   Log-Likelihood:                 2462.0
No. Observations:                 951   AIC:                            -4912.
Df Residuals:                     945   BIC:                            -4883.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0127      0.006     -1.998      0.0